# Binarized Convolutional Neural Networks

This tutorial demonstrates training a simple Binarized Convolutional Neural Network to classify MNIST digits. This simple network will achieve over 98% accuracy on the MNIST test set. Because this tutorial uses Larq and the [Keras Sequential API](https://www.tensorflow.org/guide/keras), creating and training our model will take just a few lines of code.

### Import TensorFlow and Larq

In [1]:
import tensorflow as tf
import larq as lq

### Download and prepare the MNIST dataset

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# Normalize pixel values to be between -1 and 1
train_images, test_images = train_images / 127.5 - 1, test_images / 127.5 - 1

### Create the model

This will create a simple binarized convolutional network.

In the forward pass the quantization function
$$
q(x) = \begin{cases}
    -1 & x < 0 \\\
    1 & x \geq 0
\end{cases}
$$
is used to binarize the activations and the latent full precision weights. The gradient of this function is zero almost everywhere which prevents the model from learning.

To be able to train the model the gradient is instead estimated using the Straight-Through Estimator (STE)
(essentially the binarization is replaced by a clipped identity on the backward pass):
$$
\frac{\partial q(x)}{\partial x} = \begin{cases}
    1 & \left|x\right| \leq 1 \\\
    0 & \left|x\right| > 1
\end{cases}
$$

In larq this can be done by using `input_quantizer="ste_sign"` and `kernel_quantizer="ste_sign"`.
Additionally the latent full precision weights are clipped to be between -1 and 1 using `kernel_constraint="weight_clip"`.

In [20]:
kwargs = dict(input_quantizer="ste_sign", kernel_quantizer="ste_sign", kernel_constraint="weight_clip")

model = tf.keras.models.Sequential()

model.add(lq.layers.QuantConv2D(32, (3, 3), kernel_quantizer="ste_sign", kernel_constraint="weight_clip", use_bias=False, input_shape=(28, 28, 1)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization(scale=False))

model.add(lq.layers.QuantConv2D(64, (3, 3), use_bias=False, **kwargs))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization(scale=False))

model.add(lq.layers.QuantConv2D(64, (3, 3), use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(tf.keras.layers.Flatten())

model.add(lq.layers.QuantDense(64, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(lq.layers.QuantDense(10, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(tf.keras.layers.Activation("softmax"))

 Here's the complete architecture of our model.
 
 Almost all parameter of the network a binarized (either -1 or 1). This will make the network extremly fast when deployed on a embedded device that supports binarized neural networks.

In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quant_conv2d_18 (QuantConv2D (None, 26, 26, 32)        288       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
batch_normalization_v1_20 (B (None, 13, 13, 32)        96        
_________________________________________________________________
quant_conv2d_19 (QuantConv2D (None, 11, 11, 64)        18432     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
batch_normalization_v1_21 (B (None, 5, 5, 64)          192       
_________________________________________________________________
quant_conv2d_20 (QuantConv2D (None, 3, 3, 64)          36864     
__________

### Compile and train the model

Note: This may take a few minutes depending on your system.

In [22]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, batch_size=64, epochs=6)

test_loss, test_acc = model.evaluate(test_images, test_labels)

Epoch 1/6
60000/60000 [==============================] - 72s 1ms/sample - loss: 0.6494 - acc: 0.9070
Epoch 2/6
60000/60000 [==============================] - 67s 1ms/sample - loss: 0.4760 - acc: 0.9606
Epoch 3/6
60000/60000 [==============================] - 67s 1ms/sample - loss: 0.4480 - acc: 0.9691
Epoch 4/6
60000/60000 [==============================] - 68s 1ms/sample - loss: 0.4365 - acc: 0.9718
Epoch 5/6
60000/60000 [==============================] - 68s 1ms/sample - loss: 0.4329 - acc: 0.9739
Epoch 6/6
10000/10000 [==============================] - 6s 576us/sample - loss: 0.4283 - acc

### Evaluate the model

In [23]:
print(f"Test accuracy {test_acc * 100:.2f} %")

Test accuracy 97.51 %


As you can see, our simple binarized CNN has achieved a test accuracy of over 97.5 %. Not bad for a few lines of code!